In [1]:
# Step 0.1 — Create project folder structure

import os

BASE_DIR = "crime_hotspot_project"

folders = [
    "data/raw",
    "data/processed",
    "features",
    "models",
    "evaluation",
    "visualization"
]

for folder in folders:
    os.makedirs(os.path.join(BASE_DIR, folder), exist_ok=True)

print("✅ Project folder structure created")


✅ Project folder structure created


In [2]:
# Step 0.2 — Install libraries

!pip install geopandas shapely folium hdbscan xgboost shap scikit-learn pyproj seaborn tqdm


In [4]:
# Step 0.3 — Verify imports

import pandas as pd
import numpy as np
import geopandas as gpd
import folium
import shap
import hdbscan
import xgboost as xgb

from sklearn.ensemble import RandomForestClassifier
from sklearn.calibration import CalibratedClassifierCV
from sklearn.metrics import accuracy_score

print("✅ All core libraries imported successfully")


✅ All core libraries imported successfully


In [5]:
# Step 1.2 — Create config.py

config_code = """
BASE_DIR = 'crime_hotspot_project'

RAW_DATA_PATH = f"{BASE_DIR}/data/raw/NYPD_Complaint_Data_Historic.csv"
CLEAN_DATA_PATH = f"{BASE_DIR}/data/processed/clean_crime_data.csv"
"""

with open("crime_hotspot_project/config.py", "w") as f:
    f.write(config_code)

print("✅ config.py created")


✅ config.py created


In [7]:
# Step 1 FIX — Robust CSV loading for NYPD data

import pandas as pd
from crime_hotspot_project.config import RAW_DATA_PATH

df_raw = pd.read_csv(
    RAW_DATA_PATH,
    engine="python",        # safer for messy CSVs
    on_bad_lines="skip",    # skip corrupted rows
    encoding="ISO-8859-1"   # common for NYPD files
)

print("✅ Dataset loaded successfully")
print("Shape:", df_raw.shape)


✅ Dataset loaded successfully
Shape: (2646147, 35)


In [8]:
print(df_raw.columns.tolist())


['CMPLNT_NUM', 'CMPLNT_FR_DT', 'CMPLNT_FR_TM', 'CMPLNT_TO_DT', 'CMPLNT_TO_TM', 'ADDR_PCT_CD', 'RPT_DT', 'KY_CD', 'OFNS_DESC', 'PD_CD', 'PD_DESC', 'CRM_ATPT_CPTD_CD', 'LAW_CAT_CD', 'BORO_NM', 'LOC_OF_OCCUR_DESC', 'PREM_TYP_DESC', 'JURIS_DESC', 'JURISDICTION_CODE', 'PARKS_NM', 'HADEVELOPT', 'HOUSING_PSA', 'X_COORD_CD', 'Y_COORD_CD', 'SUSP_AGE_GROUP', 'SUSP_RACE', 'SUSP_SEX', 'TRANSIT_DISTRICT', 'Latitude', 'Longitude', 'Lat_Lon', 'PATROL_BORO', 'STATION_NAME', 'VIC_AGE_GROUP', 'VIC_RACE', 'VIC_SEX']


In [9]:
# Step 1.4 — Column selection (minimal but sufficient)

KEEP_COLS = [
    "CMPLNT_FR_DT",
    "CMPLNT_FR_TM",
    "OFNS_DESC",
    "LAW_CAT_CD",
    "BORO_NM",
    "ADDR_PCT_CD",
    "Latitude",
    "Longitude"
]


df = df_raw[KEEP_COLS].copy()

print("Columns kept:")
print(df.columns.tolist())
print("Shape after selection:", df.shape)


Columns kept:
['CMPLNT_FR_DT', 'CMPLNT_FR_TM', 'OFNS_DESC', 'LAW_CAT_CD', 'BORO_NM', 'ADDR_PCT_CD', 'Latitude', 'Longitude']
Shape after selection: (2646147, 8)


In [10]:
# Step 1.5 — Timestamp creation (date + time)

df["CMPLNT_FR_DT"] = pd.to_datetime(df["CMPLNT_FR_DT"], errors="coerce")
df["CMPLNT_FR_TM"] = df["CMPLNT_FR_TM"].astype(str)

df["timestamp"] = pd.to_datetime(
    df["CMPLNT_FR_DT"].astype(str) + " " + df["CMPLNT_FR_TM"],
    errors="coerce"
)

# Drop rows with invalid timestamps
df = df.dropna(subset=["timestamp"])

print("Rows after timestamp cleaning:", df.shape[0])


Rows after timestamp cleaning: 2646096


In [13]:
# Step 1.6a — Convert coordinates to numeric

df["Latitude"] = pd.to_numeric(df["Latitude"], errors="coerce")
df["Longitude"] = pd.to_numeric(df["Longitude"], errors="coerce")

print("Latitude dtype:", df["Latitude"].dtype)
print("Longitude dtype:", df["Longitude"].dtype)


Latitude dtype: float64
Longitude dtype: float64


In [14]:
# Step 1.6b — Remove invalid coordinates (NYC bounds)

df = df.dropna(subset=["Latitude", "Longitude"])

df = df[
    (df["Latitude"].between(40.4, 41.0)) &
    (df["Longitude"].between(-74.5, -73.5))
]

print("Rows after coordinate validation:", df.shape[0])


Rows after coordinate validation: 2645818


In [12]:
# Step 1.7 — Temporal features

df["year"] = df["timestamp"].dt.year
df["iso_week"] = df["timestamp"].dt.isocalendar().week
df["day_of_week"] = df["timestamp"].dt.dayofweek
df["hour"] = df["timestamp"].dt.hour

df[["year", "iso_week", "day_of_week", "hour"]].head()


,year,iso_week,day_of_week,hour
0,2024,1,0,5
1,2024,1,0,22
2,2024,1,0,21
3,2024,1,0,17
4,2024,1,0,0


In [15]:
# Step 1.8 — Save cleaned dataset (FINAL step of Phase 1)

from crime_hotspot_project.config import CLEAN_DATA_PATH

df.to_csv(CLEAN_DATA_PATH, index=False)

print("✅ Phase 1 completed successfully")
print("Final dataset shape:", df.shape)


✅ Phase 1 completed successfully
Final dataset shape: (2645818, 13)


#PHASE 2 — Spatial Grid Construction (500m × 500m)

In [16]:
# Step 2.1 — Load clean dataset

import pandas as pd
from crime_hotspot_project.config import CLEAN_DATA_PATH

df = pd.read_csv(CLEAN_DATA_PATH)

print("Loaded clean data:", df.shape)
df.head()


Loaded clean data: (2645818, 13)


,CMPLNT_FR_DT,CMPLNT_FR_TM,OFNS_DESC,LAW_CAT_CD,BORO_NM,ADDR_PCT_CD,Latitude,Longitude,timestamp,year,iso_week,day_of_week,hour
0,2024-12-30,05:00:00,PETIT LARCENY,MISDEMEANOR,QUEENS,114.0,40.769926,-73.888860,2024-12-30 05:00:00,2024,1,0,5
1,2024-12-30,22:00:00,HARRASSMENT 2,VIOLATION,MANHATTAN,7.0,40.711274,-73.984350,2024-12-30 22:00:00,2024,1,0,22
2,2024-12-30,21:45:00,ASSAULT 3 & RELATED OFFENSES,MISDEMEANOR,BROOKLYN,78.0,40.678520,-73.983808,2024-12-30 21:45:00,2024,1,0,21
3,2024-12-30,17:00:00,CRIMINAL MISCHIEF & RELATED OF,MISDEMEANOR,QUEENS,111.0,40.740316,-73.759881,2024-12-30 17:00:00,2024,1,0,17
4,2024-12-30,00:30:00,PETIT LARCENY,MISDEMEANOR,QUEENS,105.0,40.750884,-73.717741,2024-12-30 00:30:00,2024,1,0,0


In [125]:
from shapely import wkt
import geopandas as gpd

# ✅ Correct: geometry was created in EPSG:2263
grid_gdf = gpd.GeoDataFrame(
    grid_meta,
    geometry=grid_meta["geometry"].apply(wkt.loads),
    crs="EPSG:2263"
)


In [126]:
# ✅ Convert to lat/lon for Folium
grid_gdf = grid_gdf.to_crs(epsg=4326)

print(grid_gdf.crs)


EPSG:4326


In [19]:
# Step 2.4 — Bounding box of all crime points

minx, miny, maxx, maxy = gdf.total_bounds

print("Bounding box:")
print("minx:", minx)
print("miny:", miny)
print("maxx:", maxx)
print("maxy:", maxy)


Bounding box:
minx: 913411.185518524
miny: 121281.85680385165
maxx: 1067305.973777274
maxy: 271819.6009589547


In [20]:
# Step 2.5 — Create spatial grid (500m resolution)

import numpy as np

GRID_SIZE = 500  # meters

x_coords = np.arange(minx, maxx + GRID_SIZE, GRID_SIZE)
y_coords = np.arange(miny, maxy + GRID_SIZE, GRID_SIZE)

grid_cells = []
grid_ids = []

gid = 0
for x in x_coords[:-1]:
    for y in y_coords[:-1]:
        cell = Point(x + GRID_SIZE/2, y + GRID_SIZE/2)
        grid_cells.append(cell)
        grid_ids.append(gid)
        gid += 1

grid_gdf = gpd.GeoDataFrame(
    {"grid_id": grid_ids},
    geometry=grid_cells,
    crs=gdf.crs
)

print("Total grid cells created:", len(grid_gdf))
grid_gdf.head()


Total grid cells created: 93016


,grid_id,geometry
0,0,POINT (913661.186 121531.857)
1,1,POINT (913661.186 122031.857)
2,2,POINT (913661.186 122531.857)
3,3,POINT (913661.186 123031.857)
4,4,POINT (913661.186 123531.857)


from matplotlib import pyplot as plt
_df_0['grid_id'].plot(kind='hist', bins=20, title='grid_id')
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
import seaborn as sns
def _plot_series(series, series_name, series_index=0):
  palette = list(sns.palettes.mpl_palette('Dark2'))
  counted = (series['grid_id']
                .value_counts()
              .reset_index(name='counts')
              .rename({'index': 'grid_id'}, axis=1)
              .sort_values('grid_id', ascending=True))
  xs = counted['grid_id']
  ys = counted['counts']
  plt.plot(xs, ys, label=series_name, color=palette[series_index % len(palette)])

fig, ax = plt.subplots(figsize=(10, 5.2), layout='constrained')
df_sorted = _df_1.sort_values('grid_id', ascending=True)
_plot_series(df_sorted, '')
sns.despine(fig=fig, ax=ax)
plt.xlabel('grid_id')
_ = plt.ylabel('count()')

from matplotlib import pyplot as plt
_df_2['grid_id'].plot(kind='line', figsize=(8, 4), title='grid_id')
plt.gca().spines[['top', 'right']].set_visible(False)

In [21]:
# Step 2.6 — Assign grid_id to each crime point

gdf["grid_x"] = ((gdf.geometry.x - minx) // GRID_SIZE).astype(int)
gdf["grid_y"] = ((gdf.geometry.y - miny) // GRID_SIZE).astype(int)

gdf["grid_id"] = gdf["grid_x"] * len(y_coords) + gdf["grid_y"]

print("Grid assignment completed")
gdf[["grid_id"]].head()


Grid assignment completed


,grid_id
0,61706
1,45604
2,45580
3,83197
4,90477


In [22]:
# Step 2.7 — Save grid-mapped data

GRID_DATA_PATH = "crime_hotspot_project/data/processed/crime_with_grids.csv"
GRID_META_PATH = "crime_hotspot_project/data/processed/grid_metadata.csv"

# Save crime + grid
gdf.drop(columns="geometry").to_csv(GRID_DATA_PATH, index=False)

# Save grid metadata
grid_gdf.to_csv(GRID_META_PATH, index=False)

print("✅ Phase 2 grid data saved")
print("Crime-grid file:", GRID_DATA_PATH)
print("Grid metadata file:", GRID_META_PATH)


✅ Phase 2 grid data saved
Crime-grid file: crime_hotspot_project/data/processed/crime_with_grids.csv
Grid metadata file: crime_hotspot_project/data/processed/grid_metadata.csv


#PHASE 3 — Weekly Aggregation & Lag Feature Engineering

In [23]:
# Step 3.1 — Load grid-mapped crime data

import pandas as pd

GRID_DATA_PATH = "crime_hotspot_project/data/processed/crime_with_grids.csv"

df = pd.read_csv(GRID_DATA_PATH)

print("Loaded crime-grid data shape:", df.shape)
df.head()


Loaded crime-grid data shape: (2645818, 16)


,CMPLNT_FR_DT,CMPLNT_FR_TM,OFNS_DESC,LAW_CAT_CD,BORO_NM,ADDR_PCT_CD,Latitude,Longitude,timestamp,year,iso_week,day_of_week,hour,grid_x,grid_y,grid_id
0,2024-12-30,05:00:00,PETIT LARCENY,MISDEMEANOR,QUEENS,114.0,40.769926,-73.888860,2024-12-30 05:00:00,2024,1,0,5,203,197,61706
1,2024-12-30,22:00:00,HARRASSMENT 2,VIOLATION,MANHATTAN,7.0,40.711274,-73.984350,2024-12-30 22:00:00,2024,1,0,22,150,154,45604
2,2024-12-30,21:45:00,ASSAULT 3 & RELATED OFFENSES,MISDEMEANOR,BROOKLYN,78.0,40.678520,-73.983808,2024-12-30 21:45:00,2024,1,0,21,150,130,45580
3,2024-12-30,17:00:00,CRIMINAL MISCHIEF & RELATED OF,MISDEMEANOR,QUEENS,111.0,40.740316,-73.759881,2024-12-30 17:00:00,2024,1,0,17,274,175,83197
4,2024-12-30,00:30:00,PETIT LARCENY,MISDEMEANOR,QUEENS,105.0,40.750884,-73.717741,2024-12-30 00:30:00,2024,1,0,0,298,183,90477


In [24]:
# Step 3.2 — Column sanity check

required_cols = ["grid_id", "year", "iso_week", "OFNS_DESC"]
missing = [c for c in required_cols if c not in df.columns]

print("Missing columns:", missing)


Missing columns: []


In [25]:
# Step 3.3 — Weekly aggregation per grid

weekly = (
    df
    .groupby(["grid_id", "year", "iso_week"])
    .size()
    .reset_index(name="crime_count")
)

print("Weekly panel shape:", weekly.shape)
weekly.head()


Weekly panel shape: (917936, 4)


,grid_id,year,iso_week,crime_count
0,2,2024,44,2
1,4,2022,32,2
2,4,2023,24,2
3,4,2024,21,1
4,5,2022,36,2


In [26]:
# Step 3.4 — Sort panel data

weekly = weekly.sort_values(
    ["grid_id", "year", "iso_week"]
).reset_index(drop=True)

weekly.head()


,grid_id,year,iso_week,crime_count
0,2,2024,44,2
1,4,2022,32,2
2,4,2023,24,2
3,4,2024,21,1
4,5,2022,36,2


In [27]:
# Step 3.5 — Lag features

weekly["lag_1"] = weekly.groupby("grid_id")["crime_count"].shift(1)
weekly["lag_2"] = weekly.groupby("grid_id")["crime_count"].shift(2)

weekly["avg_2wk"] = (weekly["lag_1"] + weekly["lag_2"]) / 2
weekly["trend_2wk"] = weekly["lag_1"] - weekly["lag_2"]

weekly.head()


,grid_id,year,iso_week,crime_count,lag_1,lag_2,avg_2wk,trend_2wk
0,2,2024,44,2,NaN,NaN,NaN,NaN
1,4,2022,32,2,NaN,NaN,NaN,NaN
2,4,2023,24,2,2.0,NaN,NaN,NaN
3,4,2024,21,1,2.0,2.0,2.0,0.0
4,5,2022,36,2,NaN,NaN,NaN,NaN


In [28]:
# Step 3.6 — Fill missing lag values

lag_cols = ["lag_1", "lag_2", "avg_2wk", "trend_2wk"]

weekly[lag_cols] = weekly[lag_cols].fillna(0)

weekly[lag_cols].describe()


,lag_1,lag_2,avg_2wk,trend_2wk
count,917936.000000,917936.000000,917936.000000,917936.000000
mean,2.845848,2.808358,2.793880,-0.028956
std,3.056765,3.077752,2.807772,2.521850
min,0.000000,0.000000,0.000000,-170.000000
25%,2.000000,1.000000,1.500000,-1.000000
50%,2.000000,2.000000,2.000000,0.000000
75%,4.000000,4.000000,3.000000,0.000000
max,172.000000,172.000000,91.000000,168.000000


In [29]:
# Step 3.7 — Hotspot labeling (top 10% per week)

weekly["hotspot"] = 0

for (y, w), grp in weekly.groupby(["year", "iso_week"]):
    threshold = grp["crime_count"].quantile(0.90)
    idx = grp[grp["crime_count"] >= threshold].index
    weekly.loc[idx, "hotspot"] = 1

print("Hotspot distribution:")
print(weekly["hotspot"].value_counts(normalize=True))


Hotspot distribution:
hotspot
0    0.837262
1    0.162738
Name: proportion, dtype: float64


In [30]:
# Step 3.8 — Save weekly panel dataset

PANEL_PATH = "crime_hotspot_project/data/processed/weekly_panel.csv"

weekly.to_csv(PANEL_PATH, index=False)

print("✅ Phase 3 completed")
print("Weekly panel saved at:", PANEL_PATH)
print("Final panel shape:", weekly.shape)


✅ Phase 3 completed
Weekly panel saved at: crime_hotspot_project/data/processed/weekly_panel.csv
Final panel shape: (917936, 9)


#PHASE 4 — Spatio-Temporal KDE Features (Hotspot Intensity)

In [31]:
# Step 4.1 — Load data for KDE modeling

import pandas as pd
import geopandas as gpd
import numpy as np

CRIME_GRID_PATH = "crime_hotspot_project/data/processed/crime_with_grids.csv"
GRID_META_PATH = "crime_hotspot_project/data/processed/grid_metadata.csv"
PANEL_PATH = "crime_hotspot_project/data/processed/weekly_panel.csv"

crime_df = pd.read_csv(CRIME_GRID_PATH)
panel_df = pd.read_csv(PANEL_PATH)
grid_gdf = gpd.read_file(GRID_META_PATH)

print("Crime rows:", crime_df.shape)
print("Panel rows:", panel_df.shape)
print("Grid cells:", grid_gdf.shape)


Crime rows: (2645818, 16)
Panel rows: (917936, 9)
Grid cells: (93016, 2)


In [33]:
# Step 4.2 — Reconstruct GeoDataFrame from grid metadata CSV

import geopandas as gpd
from shapely.geometry import Point

# Load grid metadata as DataFrame
grid_df = pd.read_csv(GRID_META_PATH)

# IMPORTANT: check column names
print(grid_df.columns)


Index(['grid_id', 'geometry'], dtype='object')


In [34]:
# Step 4.2a — Convert WKT to geometry

from shapely import wkt

grid_df["geometry"] = grid_df["geometry"].apply(wkt.loads)

grid_gdf = gpd.GeoDataFrame(
    grid_df,
    geometry="geometry",
    crs="EPSG:2263"  # NYC projected CRS
)

print("✅ Grid GeoDataFrame reconstructed")
print("CRS:", grid_gdf.crs)


✅ Grid GeoDataFrame reconstructed
CRS: EPSG:2263


In [35]:
# Step 4.2b — Extract centroid coordinates

grid_gdf["x"] = grid_gdf.geometry.x
grid_gdf["y"] = grid_gdf.geometry.y

grid_gdf[["grid_id", "x", "y"]].head()


,grid_id,x,y
0,0,913661.185519,121531.856804
1,1,913661.185519,122031.856804
2,2,913661.185519,122531.856804
3,3,913661.185519,123031.856804
4,4,913661.185519,123531.856804


In [36]:
# Step 4.3 — Merge grid centroids with weekly counts

kde_base = panel_df.merge(
    grid_gdf[["grid_id", "x", "y"]],
    on="grid_id",
    how="left"
)

print("KDE base shape:", kde_base.shape)
kde_base.head()


KDE base shape: (917936, 11)


,grid_id,year,iso_week,crime_count,lag_1,lag_2,avg_2wk,trend_2wk,hotspot,x,y
0,2,2024,44,2,0.0,0.0,0.0,0.0,0,913661.185519,122531.856804
1,4,2022,32,2,0.0,0.0,0.0,0.0,0,913661.185519,123531.856804
2,4,2023,24,2,2.0,0.0,0.0,0.0,0,913661.185519,123531.856804
3,4,2024,21,1,2.0,2.0,2.0,0.0,0,913661.185519,123531.856804
4,5,2022,36,2,0.0,0.0,0.0,0.0,0,913661.185519,124031.856804


In [47]:
from sklearn.neighbors import KernelDensity
import numpy as np
import pandas as pd

def compute_kde_fast(df_week, bandwidth=800, min_points=10):
    """
    Safe, fast KDE:
    - Uses only rows with crime_count > 0 AND valid coordinates
    - Returns Series aligned with df_week.index
    """

    kde_full = pd.Series(0.0, index=df_week.index)

    # STRICT filtering (this is the key fix)
    active = df_week[
        (df_week["crime_count"] > 0) &
        (df_week["x"].notna()) &
        (df_week["y"].notna())
    ]

    # If too few valid points, skip KDE
    if len(active) < min_points:
        return kde_full

    coords = active[["x", "y"]].values.astype(float)
    weights = active["crime_count"].values.astype(float)

    kde = KernelDensity(
        bandwidth=bandwidth,
        kernel="gaussian"
    )
    kde.fit(coords, sample_weight=weights)

    kde_vals = np.exp(kde.score_samples(coords))

    kde_full.loc[active.index] = kde_vals
    return kde_full


In [49]:
from tqdm import tqdm

kde_parts = []

for (y, w), grp in tqdm(kde_base.groupby(["year", "iso_week"], sort=False)):
    grp = grp.copy()
    grp["kde_raw"] = compute_kde_fast(grp)
    kde_parts.append(grp[["grid_id", "year", "iso_week", "kde_raw"]])

kde_df = pd.concat(kde_parts, ignore_index=True)

print("✅ KDE computed")
print("KDE rows:", kde_df.shape)


100%|██████████| 156/156 [57:56<00:00, 22.28s/it]

✅ KDE computed
KDE rows: (917936, 4)


In [50]:
# Step 4.6 — Normalize KDE values per week

kde_df["kde_norm"] = (
    kde_df.groupby(["year", "iso_week"])["kde_raw"]
    .transform(lambda x: (x - x.min()) / (x.max() - x.min() + 1e-9))
)

kde_df["kde_pct"] = (
    kde_df.groupby(["year", "iso_week"])["kde_norm"]
    .rank(pct=True)
)

print(kde_df[["kde_raw", "kde_norm", "kde_pct"]].describe())


            kde_raw       kde_norm        kde_pct
count  9.179360e+05  917936.000000  917936.000000
mean   2.841980e-10       0.091920       0.500085
std    2.249024e-10       0.075979       0.288675
min    0.000000e+00       0.000000       0.000153
25%    1.207611e-10       0.036792       0.250084
50%    2.254216e-10       0.071978       0.500085
75%    3.876621e-10       0.126613       0.750084
max    3.633338e-09       0.783137       1.000000


In [51]:
# Step 4.7 — Merge KDE features into weekly panel

panel_kde = panel_df.merge(
    kde_df,
    on=["grid_id", "year", "iso_week"],
    how="left"
)

print("Panel with KDE shape:", panel_kde.shape)
panel_kde[["kde_norm", "kde_pct"]].head()


Panel with KDE shape: (917936, 12)


,kde_norm,kde_pct
0,0.012614,0.050176
1,0.009777,0.009797
2,0.004048,0.001298
3,0.007191,0.005145
4,0.001427,0.010360


In [52]:
# Step 4.8 — Save final Phase 4 dataset

KDE_PANEL_PATH = "crime_hotspot_project/data/processed/weekly_panel_kde.csv"
panel_kde.to_csv(KDE_PANEL_PATH, index=False)

print("✅ Phase 4 completed")
print("Saved at:", KDE_PANEL_PATH)


✅ Phase 4 completed
Saved at: crime_hotspot_project/data/processed/weekly_panel_kde.csv


#PHASE 5 — Cluster Dynamics (DBSCAN & HDBSCAN)

In [53]:
# Step 5.1 — Load data for clustering

import pandas as pd
import geopandas as gpd
from shapely import wkt

PANEL_KDE_PATH = "crime_hotspot_project/data/processed/weekly_panel_kde.csv"
GRID_META_PATH = "crime_hotspot_project/data/processed/grid_metadata.csv"

panel = pd.read_csv(PANEL_KDE_PATH)

grid_df = pd.read_csv(GRID_META_PATH)
grid_df["geometry"] = grid_df["geometry"].apply(wkt.loads)

grid_gdf = gpd.GeoDataFrame(
    grid_df,
    geometry="geometry",
    crs="EPSG:2263"
)

grid_gdf["x"] = grid_gdf.geometry.x
grid_gdf["y"] = grid_gdf.geometry.y

print("Panel shape:", panel.shape)
print("Grid shape:", grid_gdf.shape)


Panel shape: (917936, 12)
Grid shape: (93016, 4)


In [54]:
# Step 5.2 — Merge grid coordinates into panel

panel = panel.merge(
    grid_gdf[["grid_id", "x", "y"]],
    on="grid_id",
    how="left"
)

print("After merging coordinates:", panel.shape)
panel[["x", "y"]].isna().sum()


After merging coordinates: (917936, 14)


,0
x,78
y,78


In [57]:
from sklearn.cluster import DBSCAN
import pandas as pd
import numpy as np

def run_dbscan(df_week, eps=750, min_samples=5):
    """
    Safe DBSCAN:
    - Uses only rows with crime_count > 0 and valid coordinates
    - Returns labels aligned with df_week.index
    """

    labels_full = pd.Series(-1, index=df_week.index)

    # STRICT filtering (this is the key fix)
    active = df_week[
        (df_week["crime_count"] > 0) &
        (df_week["x"].notna()) &
        (df_week["y"].notna())
    ]

    # If too few points, skip clustering
    if len(active) < min_samples:
        return labels_full

    coords = active[["x", "y"]].values.astype(float)

    db = DBSCAN(
        eps=eps,
        min_samples=min_samples,
        metric="euclidean"
    )

    cluster_labels = db.fit_predict(coords)

    # Assign labels only to valid rows
    labels_full.loc[active.index] = cluster_labels

    return labels_full


In [58]:
from tqdm import tqdm

dbscan_parts = []

for (y, w), grp in tqdm(panel.groupby(["year", "iso_week"], sort=False)):
    grp = grp.copy()
    grp["db_cluster"] = run_dbscan(grp)
    dbscan_parts.append(grp[["grid_id", "year", "iso_week", "db_cluster"]])

dbscan_df = pd.concat(dbscan_parts, ignore_index=True)

print("DBSCAN rows:", dbscan_df.shape)
dbscan_df.head()


100%|██████████| 156/156 [00:06<00:00, 23.67it/s]

DBSCAN rows: (917936, 4)


,grid_id,year,iso_week,db_cluster
0,2,2024,44,-1
1,309,2024,44,-1
2,615,2024,44,-1
3,917,2024,44,-1
4,1825,2024,44,-1


In [60]:
# Step 5.5 — DBSCAN feature engineering (INDEX-SAFE VERSION)

# 1. Is this grid part of a cluster this week?
dbscan_df["db_is_cluster"] = (dbscan_df["db_cluster"] != -1).astype(int)

# 2. Sort for temporal order
dbscan_df = dbscan_df.sort_values(
    ["grid_id", "year", "iso_week"]
).reset_index(drop=True)

# 3. Was cluster in previous week?
dbscan_df["db_was_cluster_prev"] = (
    dbscan_df.groupby("grid_id")["db_is_cluster"]
    .shift(1)
    .fillna(0)
)

# 4. Cluster age (consecutive weeks in cluster)
dbscan_df["db_cluster_age"] = (
    dbscan_df.groupby("grid_id")["db_is_cluster"]
    .transform(lambda x: x * (x.groupby((x != x.shift()).cumsum()).cumcount() + 1))
)

# 5. Cluster change indicator
dbscan_df["db_cluster_change"] = (
    dbscan_df["db_is_cluster"] - dbscan_df["db_was_cluster_prev"]
)

dbscan_df.head(10)


,grid_id,year,iso_week,db_cluster,db_is_cluster,db_was_cluster_prev,db_cluster_age,db_cluster_change
0,2,2024,44,-1,0,0.0,0,0.0
1,4,2022,32,-1,0,0.0,0,0.0
2,4,2023,24,-1,0,0.0,0,0.0
3,4,2024,21,-1,0,0.0,0,0.0
4,5,2022,36,-1,0,0.0,0,0.0
5,5,2022,48,-1,0,0.0,0,0.0
6,5,2022,52,-1,0,0.0,0,0.0
7,6,2023,3,-1,0,0.0,0,0.0
8,6,2023,28,-1,0,0.0,0,0.0
9,6,2023,29,-1,0,0.0,0,0.0


In [61]:
# Step 5.6 — Merge DBSCAN features into panel

panel = panel.merge(
    dbscan_df[
        [
            "grid_id",
            "year",
            "iso_week",
            "db_is_cluster",
            "db_was_cluster_prev",
            "db_cluster_age",
            "db_cluster_change",
        ]
    ],
    on=["grid_id", "year", "iso_week"],
    how="left"
)

print("Panel shape after DBSCAN merge:", panel.shape)

panel[
    [
        "db_is_cluster",
        "db_was_cluster_prev",
        "db_cluster_age",
        "db_cluster_change",
    ]
].head()


Panel shape after DBSCAN merge: (917936, 18)


,db_is_cluster,db_was_cluster_prev,db_cluster_age,db_cluster_change
0,0,0.0,0,0.0
1,0,0.0,0,0.0
2,0,0.0,0,0.0
3,0,0.0,0,0.0
4,0,0.0,0,0.0


In [63]:
# Step 5.7 — Save DBSCAN-enhanced panel

DBSCAN_PANEL_PATH = "crime_hotspot_project/data/processed/weekly_panel_kde_dbscan.csv"

panel.to_csv(DBSCAN_PANEL_PATH, index=False)

print("✅ Phase 5A (DBSCAN) completed")
print("Saved at:", DBSCAN_PANEL_PATH)
print("Final panel shape:", panel.shape)


✅ Phase 5A (DBSCAN) completed
Saved at: crime_hotspot_project/data/processed/weekly_panel_kde_dbscan.csv
Final panel shape: (917936, 18)


#PHASE 5B — HDBSCAN (Adaptive Hotspot Detection)

In [64]:
!pip install hdbscan


In [65]:
# Step 5B.1 — Imports

import hdbscan
import numpy as np
import pandas as pd
from tqdm import tqdm


In [66]:
# Step 5B.2 — HDBSCAN helper (safe & aligned)

def run_hdbscan(df_week, min_cluster_size=10, min_samples=5):
    """
    Adaptive clustering using HDBSCAN
    Returns labels aligned with df_week.index
    """

    labels_full = pd.Series(-1, index=df_week.index)

    # Strict filtering
    active = df_week[
        (df_week["crime_count"] > 0) &
        (df_week["x"].notna()) &
        (df_week["y"].notna())
    ]

    if len(active) < min_cluster_size:
        return labels_full

    coords = active[["x", "y"]].values.astype(float)

    clusterer = hdbscan.HDBSCAN(
        min_cluster_size=min_cluster_size,
        min_samples=min_samples,
        metric="euclidean"
    )

    cluster_labels = clusterer.fit_predict(coords)

    labels_full.loc[active.index] = cluster_labels
    return labels_full


In [67]:
# Step 5B.3 — Run HDBSCAN per week

hdbscan_parts = []

for (y, w), grp in tqdm(panel.groupby(["year", "iso_week"], sort=False)):
    grp = grp.copy()
    grp["hdb_cluster"] = run_hdbscan(grp)
    hdbscan_parts.append(
        grp[["grid_id", "year", "iso_week", "hdb_cluster"]]
    )

hdbscan_df = pd.concat(hdbscan_parts, ignore_index=True)

print("HDBSCAN rows:", hdbscan_df.shape)
hdbscan_df.head()


  0%|          | 0/156 [00:00<?, ?it/s]/usr/local/lib/python3.12/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
  1%|          | 1/156 [00:01<02:49,  1.09s/it]/usr/local/lib/python3.12/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
  1%|▏         | 2/156 [00:01<02:24,  1.07it/s]/usr/local/lib/python3.12/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning

HDBSCAN rows: (917936, 4)


,grid_id,year,iso_week,hdb_cluster
0,2,2024,44,3
1,309,2024,44,3
2,615,2024,44,3
3,917,2024,44,3
4,1825,2024,44,3


In [70]:
# Step 5B.4 — HDBSCAN feature engineering (FINAL & SAFE)

# 1. Is this grid part of a cluster?
hdbscan_df["hdb_is_cluster"] = (hdbscan_df["hdb_cluster"] != -1).astype(int)

# 2. Sort for temporal consistency
hdbscan_df = hdbscan_df.sort_values(
    ["grid_id", "year", "iso_week"]
).reset_index(drop=True)

# 3. Was this grid clustered in the previous week?
hdbscan_df["hdb_was_cluster_prev"] = (
    hdbscan_df.groupby("grid_id")["hdb_is_cluster"]
    .shift(1)
    .fillna(0)
)

# 4. Cluster age (consecutive weeks in a cluster)
hdbscan_df["hdb_cluster_age"] = (
    hdbscan_df.groupby("grid_id")["hdb_is_cluster"]
    .transform(lambda x: x * (x.groupby((x != x.shift()).cumsum()).cumcount() + 1))
)

# 5. Cluster change indicator
hdbscan_df["hdb_cluster_change"] = (
    hdbscan_df["hdb_is_cluster"] - hdbscan_df["hdb_was_cluster_prev"]
)

# Verify
hdbscan_df[
    ["hdb_is_cluster", "hdb_was_cluster_prev",
     "hdb_cluster_age", "hdb_cluster_change"]
].head()


,hdb_is_cluster,hdb_was_cluster_prev,hdb_cluster_age,hdb_cluster_change
0,1,0.0,1,1.0
1,0,0.0,0,0.0
2,0,0.0,0,0.0
3,1,0.0,1,1.0
4,1,0.0,1,1.0


In [71]:
# Step 5B.5 — Merge HDBSCAN features into panel

panel = panel.merge(
    hdbscan_df[
        [
            "grid_id",
            "year",
            "iso_week",
            "hdb_is_cluster",
            "hdb_was_cluster_prev",
            "hdb_cluster_age",
            "hdb_cluster_change",
        ]
    ],
    on=["grid_id", "year", "iso_week"],
    how="left"
)

print("Panel shape after HDBSCAN merge:", panel.shape)

panel[
    [
        "hdb_is_cluster",
        "hdb_was_cluster_prev",
        "hdb_cluster_age",
        "hdb_cluster_change",
    ]
].head()


Panel shape after HDBSCAN merge: (917936, 22)


,hdb_is_cluster,hdb_was_cluster_prev,hdb_cluster_age,hdb_cluster_change
0,1,0.0,1,1.0
1,0,0.0,0,0.0
2,0,0.0,0,0.0
3,1,0.0,1,1.0
4,1,0.0,1,1.0


In [72]:
# Step 5B.6 — Save final clustering-enhanced panel

HDBSCAN_PANEL_PATH = "crime_hotspot_project/data/processed/weekly_panel_kde_dbscan_hdbscan.csv"

panel.to_csv(HDBSCAN_PANEL_PATH, index=False)

print("✅ Phase 5B (HDBSCAN) completed")
print("Saved at:", HDBSCAN_PANEL_PATH)
print("Final panel shape:", panel.shape)


✅ Phase 5B (HDBSCAN) completed
Saved at: crime_hotspot_project/data/processed/weekly_panel_kde_dbscan_hdbscan.csv
Final panel shape: (917936, 22)


In [73]:
print("Panel shape after HDBSCAN merge:", panel.shape)


Panel shape after HDBSCAN merge: (917936, 22)


#PHASE 6 — PREDICTIVE MODELING (START)

In [74]:
# Step 6.1 — Load final dataset for modeling

import pandas as pd

DATA_PATH = "crime_hotspot_project/data/processed/weekly_panel_kde_dbscan_hdbscan.csv"

df = pd.read_csv(DATA_PATH)

print("Dataset shape:", df.shape)
df.head()


Dataset shape: (917936, 22)


,grid_id,year,iso_week,crime_count,lag_1,lag_2,avg_2wk,trend_2wk,hotspot,kde_raw,...,x,y,db_is_cluster,db_was_cluster_prev,db_cluster_age,db_cluster_change,hdb_is_cluster,hdb_was_cluster_prev,hdb_cluster_age,hdb_cluster_change
0,2,2024,44,2,0.0,0.0,0.0,0.0,0,4.517875e-11,...,913661.185519,122531.856804,0,0.0,0,0.0,1,0.0,1,1.0
1,4,2022,32,2,0.0,0.0,0.0,0.0,0,2.793335e-11,...,913661.185519,123531.856804,0,0.0,0,0.0,0,0.0,0,0.0
2,4,2023,24,2,2.0,0.0,0.0,0.0,0,2.253858e-11,...,913661.185519,123531.856804,0,0.0,0,0.0,0,0.0,0,0.0
3,4,2024,21,1,2.0,2.0,2.0,0.0,0,2.229882e-11,...,913661.185519,123531.856804,0,0.0,0,0.0,1,0.0,1,1.0
4,5,2022,36,2,0.0,0.0,0.0,0.0,0,2.796970e-11,...,913661.185519,124031.856804,0,0.0,0,0.0,1,0.0,1,1.0


In [75]:
# Step 6.2 — Time-based split (Option A)

train_df = df[df["year"] <= 2023].copy()
test_df  = df[df["year"] == 2024].copy()

print("Train shape:", train_df.shape)
print("Test shape:", test_df.shape)

print("\nTarget distribution (train):")
print(train_df["hotspot"].value_counts(normalize=True))

print("\nTarget distribution (test):")
print(test_df["hotspot"].value_counts(normalize=True))


Train shape: (612582, 22)
Test shape: (305354, 22)

Target distribution (train):
hotspot
0    0.841629
1    0.158371
Name: proportion, dtype: float64

Target distribution (test):
hotspot
0    0.828501
1    0.171499
Name: proportion, dtype: float64


In [76]:
# Step 6.3 — Define feature set

TARGET = "hotspot"

DROP_COLS = [
    "grid_id",
    "year",
    "iso_week",
    TARGET
]

FEATURES = [c for c in df.columns if c not in DROP_COLS]

print("Number of features:", len(FEATURES))
print(FEATURES)


Number of features: 18
['crime_count', 'lag_1', 'lag_2', 'avg_2wk', 'trend_2wk', 'kde_raw', 'kde_norm', 'kde_pct', 'x', 'y', 'db_is_cluster', 'db_was_cluster_prev', 'db_cluster_age', 'db_cluster_change', 'hdb_is_cluster', 'hdb_was_cluster_prev', 'hdb_cluster_age', 'hdb_cluster_change']


NaNs in X_train: 0
NaNs in X_test: 0


In [82]:
# Step 6.4 — Prepare train/test matrices

X_train = train_df[FEATURES]
y_train = train_df[TARGET]

X_test = test_df[FEATURES]
y_test = test_df[TARGET]

print("X_train:", X_train.shape)
print("X_test:", X_test.shape)


X_train: (612582, 18)
X_test: (305354, 18)


In [83]:
# Step 6.4.1 — Handle NaNs (safe & interpretable)

X_train = X_train.fillna(0)
X_test = X_test.fillna(0)

print("NaNs in X_train:", X_train.isna().sum().sum())
print("NaNs in X_test:", X_test.isna().sum().sum())


NaNs in X_train: 0
NaNs in X_test: 0


In [84]:
# Step 6.5 — Baseline Logistic Regression

from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report

baseline = LogisticRegression(
    max_iter=1000,
    class_weight="balanced",
    n_jobs=-1
)

baseline.fit(X_train, y_train)

y_pred_base = baseline.predict(X_test)

print("Baseline Logistic Regression")
print(classification_report(y_test, y_pred_base, digits=4))


Baseline Logistic Regression
              precision    recall  f1-score   support

           0     0.9238    0.9996    0.9602    252986
           1     0.9969    0.6019    0.7506     52368

    accuracy                         0.9314    305354
   macro avg     0.9604    0.8007    0.8554    305354
weighted avg     0.9364    0.9314    0.9243    305354



In [85]:
# Step 6.6 — Random Forest (main model)

from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier(
    n_estimators=300,
    max_depth=20,
    min_samples_leaf=5,
    class_weight="balanced_subsample",
    n_jobs=-1,
    random_state=42
)

rf.fit(X_train, y_train)

y_pred_rf = rf.predict(X_test)
y_prob_rf = rf.predict_proba(X_test)[:, 1]

print("Random Forest Results")
print(classification_report(y_test, y_pred_rf, digits=4))


Random Forest Results
              precision    recall  f1-score   support

           0     0.9837    0.9979    0.9908    252986
           1     0.9893    0.9200    0.9534     52368

    accuracy                         0.9846    305354
   macro avg     0.9865    0.9590    0.9721    305354
weighted avg     0.9846    0.9846    0.9843    305354



In [86]:
# Step 6.9 — Probability calibration (Sigmoid)

from sklearn.calibration import CalibratedClassifierCV

calibrated_rf = CalibratedClassifierCV(
    rf,
    method="sigmoid",
    cv=3
)

calibrated_rf.fit(X_train, y_train)

y_prob_cal = calibrated_rf.predict_proba(X_test)[:, 1]
y_pred_cal = calibrated_rf.predict(X_test)

print("Calibrated Random Forest")
print(classification_report(y_test, y_pred_cal, digits=4))


Calibrated Random Forest
              precision    recall  f1-score   support

           0     0.9659    0.9983    0.9818    252986
           1     0.9905    0.8296    0.9029     52368

    accuracy                         0.9694    305354
   macro avg     0.9782    0.9140    0.9424    305354
weighted avg     0.9701    0.9694    0.9683    305354



In [87]:
# Step 6.10 — Precision@K after calibration

p_at_5_cal = precision_at_k(y_test, y_prob_cal, k=0.05)
p_at_10_cal = precision_at_k(y_test, y_prob_cal, k=0.10)

print(f"Calibrated Precision@5%:  {p_at_5_cal:.4f}")
print(f"Calibrated Precision@10%: {p_at_10_cal:.4f}")


NameError: name 'precision_at_k' is not defined

In [88]:
# Step 6.11 — Predictive Accuracy Index (PAI)

def PAI(y_true, y_prob, k=0.05):
    n = int(len(y_prob) * k)
    idx = np.argsort(y_prob)[::-1][:n]
    return y_true.iloc[idx].sum() / (y_true.sum() * k)

pai_5 = PAI(y_test, y_prob_cal, k=0.05)
pai_10 = PAI(y_test, y_prob_cal, k=0.10)

print(f"PAI@5%:  {pai_5:.2f}")
print(f"PAI@10%: {pai_10:.2f}")


PAI@5%:  5.82
PAI@10%: 5.79


#PHASE 7 — INTERACTIVE CRIME HOTSPOT DASHBOARD

In [89]:
!pip install folium branca


In [90]:
# Step 7.1 — Imports

import folium
from folium.plugins import HeatMap
import branca.colormap as cm
import geopandas as gpd
import pandas as pd
import numpy as np


In [91]:
# Step 7.2 — Load final dataset

DATA_PATH = "crime_hotspot_project/data/processed/weekly_panel_kde_dbscan_hdbscan.csv"
GRID_PATH = "crime_hotspot_project/data/processed/grid_metadata.csv"

df = pd.read_csv(DATA_PATH)
grid_meta = pd.read_csv(GRID_PATH)

print(df.shape, grid_meta.shape)


(917936, 22) (93016, 2)


In [92]:
# Step 7.3 — Select most recent week

latest_year = df["year"].max()
latest_week = df[df["year"] == latest_year]["iso_week"].max()

print("Visualizing:", latest_year, "Week", latest_week)

df_latest = df[
    (df["year"] == latest_year) &
    (df["iso_week"] == latest_week)
].copy()

print("Rows in latest week:", df_latest.shape[0])


Visualizing: 2024 Week 52
Rows in latest week: 4658


In [113]:
# Step 7.4 — Predict probabilities for latest week (SAFE)

# Ensure FEATURES exists
assert all(col in df_latest.columns for col in FEATURES)

df_latest = df_latest.copy()

df_latest["pred_prob"] = calibrated_rf.predict_proba(
    df_latest[FEATURES].fillna(0)
)[:, 1]

# Top 5% predicted hotspots
threshold = df_latest["pred_prob"].quantile(0.95)
df_latest["pred_hotspot"] = (df_latest["pred_prob"] >= threshold).astype(int)

print(df_latest["pred_hotspot"].value_counts(normalize=True))


pred_hotspot
0    0.949979
1    0.050021
Name: proportion, dtype: float64


In [95]:
print(grid_meta.columns.tolist())


['grid_id', 'geometry']


In [114]:
# Step 7.5 — Restore geometry safely

import geopandas as gpd
from shapely import wkt

grid_gdf = gpd.GeoDataFrame(
    grid_meta,
    geometry=grid_meta["geometry"].apply(wkt.loads),
    crs="EPSG:4326"
)

print(grid_gdf.head())
print("CRS:", grid_gdf.crs)


   grid_id                          geometry
0        0  POINT (913661.18552 121531.8568)
1        1  POINT (913661.18552 122031.8568)
2        2  POINT (913661.18552 122531.8568)
3        3  POINT (913661.18552 123031.8568)
4        4  POINT (913661.18552 123531.8568)
CRS: EPSG:4326


In [97]:
grid_gdf.geometry.geom_type.value_counts()


,count
Point,93016


In [127]:
map_df = grid_gdf.merge(
    df_latest,
    on="grid_id",
    how="inner"
)

print("Map dataframe shape:", map_df.shape)


Map dataframe shape: (4657, 25)


In [116]:
# Step 7.7 — Base NYC map

import folium
from folium.plugins import HeatMap

nyc_map = folium.Map(
    location=[40.73, -73.94],
    zoom_start=11,
    tiles="CartoDB positron"
)


In [129]:
# Step 7.8 — Predicted hotspot layer (FINAL & SAFE)

hotspot_layer = folium.FeatureGroup(name="🔥 Predicted Hotspots (Top 5%)")

hotspots = map_df[map_df["pred_hotspot"] == 1]

for _, row in hotspots.iterrows():
    folium.GeoJson(
        data=row.geometry.__geo_interface__,
        style={
            "fillColor": "red",
            "color": "red",
            "weight": 0.6,
            "fillOpacity": 0.75,
        },
        tooltip=folium.Tooltip(
            f"Risk score: {row.pred_prob:.3f}"
        ),
    ).add_to(hotspot_layer)

hotspot_layer.add_to(nyc_map)

print("Hotspots added:", hotspots.shape[0])
display(nyc_map)

Output hidden; open in https://colab.research.google.com to view.

In [128]:
print(map_df.geometry.geom_type.value_counts())
print(map_df.geometry.total_bounds)


Point    4657
Name: count, dtype: int64
[-74.25027485  40.50019427 -73.70073058  40.91333772]


In [131]:
# Step 7.9 — KDE heatmap layer

kde_layer = folium.FeatureGroup(name="🌡️ KDE Intensity")

heat_data = [
    [
        geom.centroid.y,
        geom.centroid.x,
        row.kde_norm
    ]
    for geom, row in zip(map_df.geometry, map_df.itertuples())
]

HeatMap(
    heat_data,
    radius=18,
    blur=25,
    max_zoom=13
).add_to(kde_layer)

kde_layer.add_to(nyc_map)



In [132]:
display(nyc_map)

Output hidden; open in https://colab.research.google.com to view.

In [120]:
# Step 7.10 — HDBSCAN clusters layer

hdb_layer = folium.FeatureGroup(name="🔵 HDBSCAN Clusters")

clusters = map_df[map_df["hdb_is_cluster"] == 1]

for _, row in clusters.iterrows():
    folium.GeoJson(
        data=row.geometry.__geo_interface__,
        style={
            "fillColor": "blue",
            "color": "blue",
            "weight": 0.4,
            "fillOpacity": 0.4,
        },
        tooltip=folium.Tooltip(
            f"Cluster age: {int(row.hdb_cluster_age)}"
        ),
    ).add_to(hdb_layer)

hdb_layer.add_to(nyc_map)


In [121]:
# Step 7.11 — Finalize map

# Auto-zoom to data
bounds = map_df.geometry.total_bounds
nyc_map.fit_bounds([[bounds[1], bounds[0]], [bounds[3], bounds[2]]])

# Layer control
folium.LayerControl(collapsed=False).add_to(nyc_map)

# Save
import os
os.makedirs("crime_hotspot_project/output", exist_ok=True)

MAP_PATH = "crime_hotspot_project/output/nyc_crime_hotspot_dashboard.html"
nyc_map.save(MAP_PATH)

print("✅ Dashboard saved at:", MAP_PATH)


✅ Dashboard saved at: crime_hotspot_project/output/nyc_crime_hotspot_dashboard.html


In [122]:
from IPython.display import IFrame

IFrame(
    src=MAP_PATH,
    width=1000,
    height=700
)


In [123]:
print(hotspots.shape)
print(hotspots.geometry.geom_type.value_counts())
print(hotspots.geometry.total_bounds)


(233, 25)
Point    233
Name: count, dtype: int64
[ 937161.18551852  121531.85680385 1052661.18551852  271531.85680385]
